# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv()

True

In [3]:
sys.version

'3.13.5 (main, Jun 11 2025, 15:36:57) [Clang 17.0.0 (clang-1700.0.13.3)]'

## Create connection to AI Foundry Project & instantiate a project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Instantiate AGENT 1 (Q&A)

In [5]:


agent1 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Q&A",
    instructions="You are an AI agent.",
)
print(f"Created agent, agent ID: {agent1.id}")

Created agent, agent ID: asst_NUS2Pmu4JmryG0qB6y8JFhwx


## Instantiate AGENT 2 (Translator)

In [6]:


agent2 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Translator",
    instructions="You are an AI agent that translates English into Spanish.",
)
print(f"Created agent, agent ID: {agent2.id}")

Created agent, agent ID: asst_nfJGxv9YcWvNrRrX7pJi2ICI


## Instantiate AGENT 3 (Emojis creator)

In [7]:


agent3 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Emojis",
    instructions="You are an AI agent that add emojis to a text",
)
print(f"Created agent, agent ID: {agent3.id}")


Created agent, agent ID: asst_TCuM1RYCpbKi5PwMU2CqO32z


### Show all agents

In [8]:
print(agent1)
print(agent2)
print(agent3)


{'id': 'asst_NUS2Pmu4JmryG0qB6y8JFhwx', 'object': 'assistant', 'created_at': 1750172876, 'name': 'Q&A', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_nfJGxv9YcWvNrRrX7pJi2ICI', 'object': 'assistant', 'created_at': 1750172877, 'name': 'Translator', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that translates English into Spanish.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_TCuM1RYCpbKi5PwMU2CqO32z', 'object': 'assistant', 'created_at': 1750172877, 'name': 'Emojis', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


### Create a thread for AGENT 1

In [9]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_PHLqPVg3VROKOUxWz1xWZO4Y


### Create a message for AGENT 1

In [10]:

# [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content="Tell me about the 'Palacio de Bellas Artes' in Mexico City.")
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_FvGRbAOTEdcCQzqsvUGsKRUq


### Run AGENT 1

In [11]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent1.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")


Created run, run ID: run_hroJHQUB5EK1BfpBIonyvnNq


### Check status of run for AGENT 1

In [12]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


### Show usage (tokens)

In [13]:
run.usage

{'prompt_tokens': 41, 'completion_tokens': 422, 'total_tokens': 463, 'prompt_token_details': {'cached_tokens': 0}}

### Show me the stack of messages so far

In [14]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")





Message ID: msg_TrE97IBTaTBKT9Yb6acgSPYt, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic cultural landmarks in **Mexico City**, renowned for its stunning architecture and deep artistic significance.\n\n### History & Construction\n- **Construction** began in 1904 under the direction of Italian architect *Adamo Boari*, who envisioned a grand palace combining European styles.\n- The project was plagued by delays (notably the Mexican Revolution and unstable soil), and was finally completed in **1934** by Mexican architect *Federico Mariscal*.\n\n### Architecture\n- **Exterior:** The building features an impressive *neoclassical* and *Art Nouveau* façade clad in white Italian marble.\n- **Interior:** Inside, it showcases an *Art Deco* style with Mexican motifs, elaborate details, and beautiful murals by celebrated artists such as **Diego Rivera**, **David Alfaro Siqueiros**, and **Rufino 

### Show last message of AGENT 1

In [15]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic cultural landmarks in **Mexico City**, renowned for its stunning architecture and deep artistic significance.\n\n### History & Construction\n- **Construction** began in 1904 under the direction of Italian architect *Adamo Boari*, who envisioned a grand palace combining European styles.\n- The project was plagued by delays (notably the Mexican Revolution and unstable soil), and was finally completed in **1934** by Mexican architect *Federico Mariscal*.\n\n### Architecture\n- **Exterior:** The building features an impressive *neoclassical* and *Art Nouveau* façade clad in white Italian marble.\n- **Interior:** Inside, it showcases an *Art Deco* style with Mexican motifs, elaborate details, and beautiful murals by celebrated artists such as **Diego Rivera**, **David Alfaro Siqueiros**, and **Rufino Tamayo**.\n- The palace is crowned with a spectacular **stained gla

### SHow bottom of stack of messages

In [16]:
if messages:
    last_message = messages[-1]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Tell me about the 'Palacio de Bellas Artes' in Mexico City.", 'annotations': []}}]


### Run AGENT 2

In [17]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent2.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_6VdZ5vSBcnqJMLryjGUuk7Mb


### Check Status of the run for AGENT 2

In [19]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.COMPLETED


In [20]:
run.usage

{'prompt_tokens': 470, 'completion_tokens': 243, 'total_tokens': 713, 'prompt_token_details': {'cached_tokens': 0}}

### SHow the stack of messages so far

In [21]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")



Message ID: msg_6DPtYm0rGgwE0qXhFoigX441, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los edificios más emblemáticos de la Ciudad de México y un importante centro cultural. Su construcción comenzó en 1904 y fue concluida en 1934. El palacio destaca por su impresionante arquitectura, que combina los estilos Art Nouveau en el exterior y Art Decó en el interior. Está cubierto de mármol blanco y su cúpula colorida es uno de los referentes visuales de la ciudad.\n\nEn el interior, el palacio alberga murales de artistas reconocidos como Diego Rivera, David Alfaro Siqueiros y Rufino Tamayo. También cuenta con un telón de cristal realizado por Tiffany & Co., que representa el Valle de México.\n\nEl Palacio de Bellas Artes es sede de la Ópera Nacional, el Ballet Folklórico de México y la Orquesta Sinfónica Nacional. Además, acoge exposiciones de arte, conciertos, obras de teatro y otros eventos culturales de gran relevancia.\n\

### SHow last message of AGENT 2

In [22]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los edificios más emblemáticos de la Ciudad de México y un importante centro cultural. Su construcción comenzó en 1904 y fue concluida en 1934. El palacio destaca por su impresionante arquitectura, que combina los estilos Art Nouveau en el exterior y Art Decó en el interior. Está cubierto de mármol blanco y su cúpula colorida es uno de los referentes visuales de la ciudad.\n\nEn el interior, el palacio alberga murales de artistas reconocidos como Diego Rivera, David Alfaro Siqueiros y Rufino Tamayo. También cuenta con un telón de cristal realizado por Tiffany & Co., que representa el Valle de México.\n\nEl Palacio de Bellas Artes es sede de la Ópera Nacional, el Ballet Folklórico de México y la Orquesta Sinfónica Nacional. Además, acoge exposiciones de arte, conciertos, obras de teatro y otros eventos culturales de gran relevancia.\n\nUbicado en el centro histórico, junto a la Alameda Central, el Pal

### Run AGENT 3

In [23]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent3.id)
 # [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_VaomWxeamQc1W50PF0vouTWG


### Check status of run of AGENT 3

In [24]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [25]:
run.usage

{'prompt_tokens': 715, 'completion_tokens': 161, 'total_tokens': 876, 'prompt_token_details': {'cached_tokens': 0}}

### Show the Stack of messages so far

In [26]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_MEnsmHiNzsQ9mRo8MKyEcQNK, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'Of course! Here’s a version with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is one of the most iconic landmarks in **Mexico City** 🇲🇽. Known for its stunning architecture, it combines Art Nouveau 🦋 and Art Deco ✨ styles. The building features a beautiful marble façade 🤍 and a magnificent stained-glass dome 🌈.  \n\nInside, you’ll find impressive murals 🎨 by famous Mexican artists like **Diego Rivera** and **David Alfaro Siqueiros**. The palace hosts opera 🎶, ballet 🩰, and various cultural events, making it a vibrant center for the arts.  \n\nLocated by the Alameda Central park 🌳, it’s a must-visit spot for anyone exploring the city!', 'annotations': []}}]
Message ID: msg_6DPtYm0rGgwE0qXhFoigX441, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los edificios más emblemáticos de la Ciudad de México y un importa

### Print last message of AGENT 3, this shows all Agents work combined in final output!!

In [27]:
messages = list(project_client.messages.list(thread_id=thread.id))

last_message = messages[0]
print(f"Content: {last_message.content}")




Content: [{'type': 'text', 'text': {'value': 'Of course! Here’s a version with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is one of the most iconic landmarks in **Mexico City** 🇲🇽. Known for its stunning architecture, it combines Art Nouveau 🦋 and Art Deco ✨ styles. The building features a beautiful marble façade 🤍 and a magnificent stained-glass dome 🌈.  \n\nInside, you’ll find impressive murals 🎨 by famous Mexican artists like **Diego Rivera** and **David Alfaro Siqueiros**. The palace hosts opera 🎶, ballet 🩰, and various cultural events, making it a vibrant center for the arts.  \n\nLocated by the Alameda Central park 🌳, it’s a must-visit spot for anyone exploring the city!', 'annotations': []}}]


## Post processing

In [28]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_TCuM1RYCpbKi5PwMU2CqO32z, Name: Emojis, Model: gpt-4.1, Instructions: You are an AI agent that add emojis to a text
Agent ID: asst_nfJGxv9YcWvNrRrX7pJi2ICI, Name: Translator, Model: gpt-4.1, Instructions: You are an AI agent that translates English into Spanish.
Agent ID: asst_NUS2Pmu4JmryG0qB6y8JFhwx, Name: Q&A, Model: gpt-4.1, Instructions: You are an AI agent.


In [ ]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")
        break

In [ ]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")